In [ ]:
import re
import subprocess
import os
import json

import pandas as pd

# import xlwings as xw


In [ ]:
exec_name = "gaiad"
dir = "out/" + exec_name + "/"
dir_md = dir + "comands-md-files/"


In [ ]:
if not os.path.isdir(dir):
    os.mkdir(dir)
    os.mkdir(dir_md)

for f in os.listdir(dir_md):
    os.remove(os.path.join(dir_md, f))


In [ ]:
chapters_to_include = [
    "Available Commands:",
    "Usage:",
    "Aliases:",
    "Flags:",
    "Global Flags:",
    "Example:",
    "Examples:",
]


In [ ]:
def get_cli_output(command: str):

    _args = command.split()
    _args.append("-h")

    out = subprocess.run(_args, capture_output=True)
    text = out.stdout.decode("utf-8") + out.stderr.decode("utf-8")

    with open(dir_md + command + ".md", "w") as file:
        file.write(text)

    return text


# text = get_cli_output('cyber')


In [ ]:
def parse_cli_stdout(command: str):

    _text = get_cli_output(command)
    _lines = _text.split("\n")

    chapters = {"command": command}
    chapter = "Description"
    chapters[chapter] = []

    for l in _lines:
        if l != "":
            if l in chapters_to_include:
                chapter = l[:-1]
                chapters[chapter] = []
            else:
                chapters[chapter].append(l)

    return chapters


# chapters = parse_cli_stdout("gaiad")
# print(chapters)


In [ ]:
def get_sub_commands(chapters):
    new_commands = []

    if "Available Commands" in chapters:
        for l in chapters["Available Commands"]:
            if _match := re.search(r"^\s+(.*?)\s+(.*)", l):
                sub_command = _match.group(1)

                command_candidate = " ".join(
                    (chapters["command"] + " " + sub_command).split()
                )
                new_commands.append(command_candidate)

    return new_commands
    # del chapters["Available Commands"]


# new_commands = get_sub_commands(chapters)
# new_commands


In [ ]:
parsed_commands = []
all_commands = []
new_commands = [exec_name + " "]

while True:
    for command in new_commands:

        commands_dict = parse_cli_stdout(command)
        new_command_candidates = get_sub_commands(commands_dict)
        all_commands.append(commands_dict)

        for candidate in new_command_candidates:
            if candidate not in parsed_commands:
                new_commands.append(candidate)
                parsed_commands.append(candidate)
        new_commands.remove(command)
    if new_commands == []:
        break


In [ ]:
# all_commands


In [ ]:
json_object = json.dumps(all_commands, indent=4)

with open(dir + exec_name + "_commands.json", "w") as outfile:
    outfile.write(json_object)


In [ ]:
len(all_commands)


In [ ]:
command_files = []

for f in os.listdir(dir_md):
    if f[-3:] == ".md":
        command_files.append(f)

command_files.sort()
# command_files


In [ ]:
if os.path.exists(dir + exec_name + "_commands.md"):
    os.remove(dir + exec_name + "_commands.md")

for f in command_files:
    with open(dir_md + f) as f2:
        text = f2.read()
        file_content = f"### {f[:-3]}\n\n```\n{text}\n```\n\n"

        with open(dir + exec_name + "_commands.md", "a") as file:
            file.write(file_content)


In [ ]:
# with open("all_commands.json", "r") as file:
#     all_commands = json.load(file)


In [ ]:
commands = []

for c in all_commands:
    if "Available Commands" not in c:
        commands.append(c)


In [ ]:
count = 0
for i in commands:
    all_flags = []
    all_flags.extend(i["Flags"])
    del i["Flags"]
    if "Global Flags" in i:
        all_flags.extend(i["Global Flags"])
        del i["Global Flags"]

    if all_flags != []:
        i["flags_parsed"] = []
        for b in all_flags:
            if match1 := re.match(
                r"\s\s+(?:-(?P<short>\w)[,\s]+)?(?:--(?P<long>[.\w-]+))\s(?:(?P<format>(uint32|ints|strings|int 64Slice|int|string|uint|float|bytesHex)))?\s*(?P<description>.*)?",
                b,
            ):
                i["flags_parsed"].append(
                    {
                        "long": match1.group("long"),
                        "short": match1.group("short"),
                        "format": match1.group("format"),
                        "description": match1.group("description"),
                    }
                )


In [ ]:
for c in commands:
    # c["flags_parsed"] = [f for f in c["flags_parsed"] if f["long"] != "help"]
    c["Usage"] = [
        l.replace("[flags]", "").replace(c["command"], "").strip() for l in c["Usage"]
    ]
    c["Description"] = " ".join(c["Description"])[:200]


In [ ]:
for c in commands:
    clean_string = c["Usage"][0].split("(")[0].split("--")[0]
    args = re.findall(r"(?:\[|\<)(\w.+?)(?:\]|\>)", clean_string)
    c["args"] = []
    for p in args:
        p = re.sub(r"[^A-Za-z0-9]", "_", p)
        c["args"].append(p)


In [ ]:
# df = pd.DataFrame(commands)
# pd.json_normalize(df['flags_parsed'])


In [ ]:
# df = pd.DataFrame(commands).explode('flags_parsed').reset_index()
# df['command'] = df['command'].apply(lambda x: str(x))
# df2 = pd.json_normalize(df['flags_parsed'])

# df3 = pd.concat([df[['command', 'Usage']], df2], axis=1)
# xw.view(df3)


In [ ]:
# for_excel = pd.DataFrame(data=commands).applymap(lambda x: str(x))
# xw.view(for_excel)


In [ ]:
# for i in all_commands:
#     print(i["command"], "\n", i["flags_parsed"])


In [ ]:
types_nu_dict = {
    "uint": "int",
    "uint32": "int",
    # "float": "decimal",
    "float": "string",
    "strings": "string",
    "bytesHex": "string",
    "ints": "string",
}
lines = []

for c in commands:
    lines.append(f"\n# {c['Description']}\nextern '{c['command']}' [\n")
    for p in c["args"]:
        lines.append(f"\t{p}: string\n")
    # print(f"\n# {c['Description']}\nextern '{c['command']}' [")
    for f in c["flags_parsed"]:
        if "." not in f["long"]:
            long = f["long"]
            short = "(-" + f["short"] + ")" if f["short"] else ""
            f1 = types_nu_dict.get(f["format"], f["format"])
            format = ": " + f1 if f1 else ""
            description = f["description"]
            lines.append(f"\t--{long}{short}{format}\t\t# {description}\n")
            # print(f"\t--{long}{short}{format}\t\t# {description}")
    lines.append("]")
    # print("]")

with open(dir + exec_name + "_nu_completions.nu", "w") as file:
    file.writelines(lines)
